In [1]:
import numpy as np
import re
import random
np.random.seed(420)

Para generar una palabra se necesitan diversas categorías. Por ejemplo, las vocales podrían ser V=aeiou, y las consonantes C=bcdfghijklmnpqrstvwxyz. Para que el programa funcione los nombres de las categorías deben ser una letra mayúscula. El primer argumento de ```generate_word()``` es un diccionario cuyas claves son el nombre de la clase y el argumento una lista con los elementos de dicha clase.

También se necesita saber de qué forma están distribuidas esas clases. Esto corresponde al segundo argumento de la función, que podría ser, por ejemplo, 'CVC' es decir: una consonante seguida de una vocal y luego una consonante, o 'C(y)V" es decir, una consonante y una vocal, pero entre estas puede ir una 'y'. O incluso 'V\[n,m\]V', que quiere decir una vocal seguida de n o m, seguida de otra vocal.

El tercer argumento es el número de sílabas.

El cuarto está relacionado con la frecuencia con la que aparecen las palabras. Como ejemplo, en el español la letra más común es la 'E' (13.72%), luego le sigue la 'A' (11.72%), luego la 'O' (8.44%) y así sucesivamente. El cuarto argumento dice qué tanto baja la frecuencia. Por ejemplo, si es 0.5, la primera letra de una clase va a aparecer el 50% de las veces, la segunda el 25% y así. Si este argumento es 0, todas las palabras tienen la misma probabilidad de ser escogidas.

In [2]:
def generate_word(alphabet={
    'C': list('sdntkf'),
    'V': list('eaou') 
}, constraint='CVC', syllables=3, dropoff=0.2):
    word = ''
    for i in range(syllables):
        word += generate_syllable(alphabet, constraint, dropoff)
    return word

def generate_syllable(alphabet, constraint, dropoff):
    constraint = resolve_optionals(constraint)
    constraint = select_from_lists(constraint)
    syllable = ''
    for element in constraint:
        letter = ''
        if element.isupper():
            choices = alphabet[element]
            if dropoff == 0:
                letter = np.random.choice(choices)
            else:    
                p = generate_weights(len(choices), dropoff)
                letter = np.random.choice(choices, p=p)
        else:
            letter = element
        syllable += letter
    return syllable

def resolve_optionals(constraint):
    while "(" in constraint:        
        constraint = resolve_optional(constraint)
    return constraint
        
def resolve_optional(constraint):
    regex = r"\(([^\(]*?)\)"
    result = re.search(regex, constraint)
    if bool(result):
        whole = result.group(0)
        content = result.group(1)
        if np.random.randint(0, 5) == 0:
            constraint = constraint.replace(whole, content)
        else:
            constraint = constraint.replace(whole, '')
            
    return constraint

def select_from_lists(constraint):
    regex = r'\[(([a-z],)*[a-z])\]'
    result = re.search(regex, constraint)
    if bool(result):
        whole = result.group(0)
        content = result.group(1).split(',')
        chosen_one = np.random.choice(content)
        constraint = constraint.replace(whole, chosen_one)
    return constraint

def generate_weights(i, alpha):
    w = (1 - alpha) ** np.arange(i)[::-1]
    w /= w.sum()
    return np.flip(w)

generate_word(syllables=3, constraint="CV")

'daseko'

Para generar una oración hay que pegar un montón de palabras juntas. La primera letra de cada una comienza con mayúscula y termina en un signo de puntuación. Aquí cada palabra puede tener un patrón distinto. Por ejemplo, CVC para una, o CV(V) para otras.

Las palabras generadas por cada patrón pueden tener diferentes tamaños. La probabilidad de que una palabra tenga longitud $i$ está dada por $P_i$. El segundo argumento para esta función es un diccionario donde las llaves son el patrón y el valor es un array representando a $P$.

Por ejemplo: ```CVC: [0.1, 0.8, 0.1]``` quiere decir que las sílabas de una palabra tienen el patrón CVC, y que hay un 10% de probabilidades de que sea de una sola sílaba, 80% de probabilidades de que sea de dos sílabas y 10% de que la palabra tenga tres sílabas.

Un párrafo solo es un montón de oraciones.

In [3]:
def generate_sentence(alphabet={
    'C': list('sdntkf'),
    'V': list('eaou') 
}, syllable_lengths={'CV(V)': [0, 1, 0]}, dropoff=0.2, max_words=10):
    number_of_words = np.random.randint(1, max_words)
    puntuation = list('.?!')
    constraints = list(syllable_lengths.keys())
    sentence = ''
    for i in range(number_of_words):
        constraint, probabilities = random.choice(list(syllable_lengths.items()))
        lengths = range(1, len(probabilities)+1)
        length = np.random.choice(lengths, p=probabilities)
        word = generate_word(syllables=length, constraint=constraint, alphabet=alphabet, dropoff=dropoff)
        sentence += f"{word} "
    sentence = sentence.strip().capitalize()
    sentence += np.random.choice(puntuation, p=[0.8, 0.1, 0.1])
    return sentence

def generate_paragraph(alphabet={
    'C': list('sdntkf'),
    'V': list('eaou') 
}, syllable_lengths={'CV(V)': [0, 1, 0]}, dropoff=0.2, max_sentences=10):
    paragraph = ''
    for i in range(max_sentences):
        sentence = generate_sentence(alphabet, syllable_lengths, dropoff=dropoff) + " "
        paragraph += sentence
    return paragraph.rstrip()

In [14]:
from IPython.core.display import HTML

alphabet = {
    'C': list('sdntkfbgh'),
    'V': list('eaou')
}

syllable_lengths={
    'VC(V)': [0.1, 0.5, 0.4],
    'CV': [0.2, 0.8]
}

def generate_title():
    return generate_sentence(alphabet, syllable_lengths, max_words=3)[:-1]

HTML("""
<style>
.document {
    font-size: medium;
    font-family: serif;
}
</style>
<div class="document">
<h1>"""+generate_title()+"""</h1>
<p> """ +generate_paragraph(alphabet, syllable_lengths)+ """</p>
<p> """ +generate_paragraph(alphabet, syllable_lengths)+ """</p>
<h2>"""+generate_title()+"""</h2>
<p> """ +generate_paragraph(alphabet, syllable_lengths)+ """</p>
<p> """ +generate_paragraph(alphabet, syllable_lengths)+ """</p>
</div>
""")